In [1]:
import cv2
import os
import numpy as np

# Function to preprocess the image (grayscale, invert, threshold, and apply controlled morphological operations)
def pre_process_image(img_file):
    image = cv2.imread(img_file)
    if image is None:
        raise ValueError("Image could not be loaded. Please check the file path.")
    
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_img = cv2.bitwise_not(gray_img)
    
    # Threshold the image using Otsu's method
    threshold_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # Define a smaller kernel for morphological operations
    kernel = np.ones((2, 2), np.uint8)  # A smaller 2x2 kernel for subtle effects

    # Apply morphological closing with a small kernel to fill small gaps
    closed_img = cv2.morphologyEx(threshold_img, cv2.MORPH_CLOSE, kernel)

    # Apply dilation with a smaller kernel to connect small gaps in symbols, minimal iterations
    dilated_img = cv2.dilate(closed_img, kernel, iterations=1)

    return dilated_img

# Function to crop and save individual characters
def crop_letter(image, start_row, end_row, start_col, end_col, line_number, character_id):
    cropped_char = image[start_row:end_row, start_col:end_col]
    char_name = f'lines/line{line_number}/line_chars/char{character_id}.png'
    cv2.imwrite(char_name, cropped_char)

# Function to crop and save individual lines, also calls segmentation of characters
def crop_line(image, start_row, end_row, start_col, end_col, line_number):
    cropped_line = image[start_row:end_row, start_col:end_col]
    line_path = f'lines/line{line_number}'
    os.makedirs(f'{line_path}/line_chars', exist_ok=True)
    line_name = f'{line_path}/line.png'
    cv2.imwrite(line_name, cropped_line)
    
    segment_in_letters(image, start_row, end_row, start_col, end_col, cropped_line, line_number)

# Function to segment a line into individual letters, includes letter tracking logic
def segment_in_letters(image, start_row, end_row, start_col, end_col, cropped_line, line_number):
    character_id = 0
    col = 0
    line_rows = end_row - start_row
    line_cols = end_col - start_col

    while col < line_cols:
        if np.sum(cropped_line[:, col]) > 0:  # Non-empty column (letter detected)
            start_letter_col = col

            # Track the end of the letter
            for c in range(start_letter_col, line_cols):
                if np.sum(cropped_line[:, c]) == 0:  # Empty column detected
                    end_letter_col = c
                    break
            else:
                end_letter_col = line_cols  # If no empty column found, set to end of line

            # Crop and save the detected letter
            crop_letter(image, start_row, end_row, start_letter_col, end_letter_col, line_number, character_id)
            character_id += 1
            col = end_letter_col
        else:
            col += 1

# Function to segment the entire image into lines, includes line tracking logic
def segment_in_lines(image):
    rows, cols = image.shape
    row = 0
    line_number = 0

    while row < rows:
        if np.sum(image[row, :]) > 0:  # Non-empty row (line detected)
            start_row = row

            # Track the end of the line
            for r in range(start_row, rows):
                if np.sum(image[r, :]) == 0:  # Empty row detected
                    end_row = r
                    break
            else:
                end_row = rows  # If no empty row found, set to end of image

            # Crop and save the detected line
            crop_line(image, start_row, end_row, 0, cols, line_number)
            line_number += 1
            row = end_row
        else:
            row += 1

# Main function to call the processing and segmentation
def main(img_file):
    image = pre_process_image(img_file)
     
    segment_in_lines(image)

if __name__ == '__main__':
    main('test11.jpg')